In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('rural.csv')

In [3]:
df.columns

Index(['auction_id', 'round', 'bidder', 'frn', 'item', 't+l_weight', 'tier',
       'latency', 'price_point_bid', 'min_scale_pct', 'implied_support_bid',
       'implied_support_clock_pct', 'implied_support_capped', 'state',
       'county', 'census_id', 'locations', 'bid_id', 'selection_number'],
      dtype='object')

In [4]:
print(f"Dataset size: {len(df)}")

Dataset size: 4331247


In [6]:
print(f"Auction ID: {set(df['auction_id'])}, Rounds: {len(set(df['round']))}, Items: {len(set(df['item']))}, Bidders: {len(set(df['bidder']))}")

Auction ID: {904}, Rounds: 19, Items: 61766, Bidders: 316


In [7]:
df = df.sort_values(by=['round'], ascending=True)

In [8]:
df.iloc[200]

auction_id                                                   904
round                                                          1
bidder                       AMG Technology Investment Group LLC
frn                                                     21701891
item                                              MN-163-0704052
t+l_weight                                                     0
tier                                                     Gigabit
latency                                                      Low
price_point_bid                                            180.0
min_scale_pct                                                NaN
implied_support_bid                                       3230.0
implied_support_clock_pct                                 3230.0
implied_support_capped                                         Y
state                                                         MN
county                                                Washington
census_id                

## Auction Implementation

In [9]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuctionWithData

logging.basicConfig(level=20)

In [10]:
device = 'cuda'

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [14]:
bidders = list(set(df['bidder']))
items = list(set(df['item']))
rounds = df['round'].max()

In [23]:
bidders

['Reservation Telephone Cooperative',
 'Consortium 2020',
 'Wabash Communications, Inc.',
 'Montana Internet Corporation',
 'South Arkansas Telephone Company',
 'Farmers Mutual Telephone Company',
 'FiberLight, LLC',
 'Ultimate Internet Access, Inc.',
 'Rowe Wireless Networks LLC',
 'HomeTown Broadband, Inc.',
 'Citizens Vermont Acquisition Corporation',
 'St Paul Cooperative Telephone Association',
 'Quantum Telecommunications, Inc.',
 'Atlantic Telephone Membership Corporation',
 'Altice USA, Inc.',
 'Atlantic Broadband Finance, LLC',
 'Aptitude Internet LLC',
 'Rocket Connect LLC',
 'Digital Path, Inc.',
 'Palmetto Telephone Communications, LLC',
 'Horry Telephone Cooperative, Inc.',
 'Kingdom Telecommunications, Inc.',
 'Scott County Telephone Cooperative, Inc.',
 'Hankins Information Technology',
 'Somerset Telephone Co., Inc.',
 'Reedsburg Utility Commission',
 'Northwoods Communication Technologies, LLC',
 'Mountain West Technologies Corporation',
 'Centre WISP Venture Company, 

In [27]:
df.loc[df['bidder'] == 'Space Exploration Technologies Corp.'].groupby(['round']).agg({'item': lambda x: ','.join(x), 'implied_support_bid': 'sum'})

,item,implied_support_bid
round,,
1,"AR-059-0205003,AR-059-0205002,AR-059-0205001,A...",1.599998e+09
2,"ID-055-0019002,AL-059-9730001,AL-059-9729003,A...",1.599998e+09
3,"AL-099-0762001,AL-099-0761005,AL-099-0761004,A...",1.599998e+09
4,"MO-025-9501002,MO-025-9501001,MO-023-9509004,M...",1.599998e+09
5,"ME-021-9605001,ME-021-9604002,ME-021-9604001,M...",1.599998e+09
6,"IN-125-9542004,IN-125-9542003,IN-125-9541003,I...",1.599998e+09
7,"GA-185-0108002,GA-185-0106043,GA-185-0106042,G...",1.599998e+09
8,"MA-009-2621002,MA-009-2611022,MA-009-2611021,M...",1.599998e+09
9,"AL-003-0109041,AL-003-0109033,AL-003-0109032,A...",1.582220e+09


In [11]:
dataset = []
value_functions = []
for i, bidder in enumerate(bidders):
    ds = df.loc[df['bidder'] == bidder].groupby(['round']).agg({'implied_support_bid': 'sum'}).join(pd.get_dummies(df['item']))
    X, y = torch.tensor(ds[ds.columns[1:]].values.tolist()).float(), torch.tensor(ds['implied_support_bid'].values.tolist()).float().unsqueeze(1)
    items = list(ds.columns[1:])
    dataset.append((X, y))
    print(f'{i}, Bidder: {bidder}:', X.shape, y.shape, torch.max(y).numpy(), len(items))
    value_functions.append(DSFValueFunction(items, torch.max(y).numpy(), [5, 6, 3], [5000, 3000, 2000]))

MemoryError: Unable to allocate 249. GiB for an array with shape (61766, 4331247) and data type uint8

In [31]:
auction = CombinatorialAuctionWithData(bidders, items, dataset, value_functions)
allocations, payments = auction.run(epochs=1000, lr=0.001, delta=0.005, sample_rate=40)

INFO:root:Allocation calculation...
INFO:root:Bidder 1, loss: 159473991680.0
INFO:root:Bidder 2, loss: 3037593876824064.0
INFO:root:Bidder 3, loss: 12707663183872.0
INFO:root:Bidder 4, loss: 6048503037952.0
INFO:root:Bidder 5, loss: 605673344.0
INFO:root:Bidder 6, loss: 391172960.0
INFO:root:Bidder 7, loss: 437623783424.0
INFO:root:Bidder 8, loss: 1598154780704768.0
INFO:root:Bidder 9, loss: 45894164480.0
INFO:root:Bidder 10, loss: 1924135780352.0
INFO:root:Bidder 11, loss: 2190402560.0
INFO:root:Bidder 12, loss: 401025888.0
INFO:root:Bidder 13, loss: 13405801472.0
INFO:root:Bidder 14, loss: 245941968.0
INFO:root:Bidder 15, loss: 726014784.0
INFO:root:Bidder 16, loss: 246352822272.0
INFO:root:Bidder 17, loss: 118993305600.0
INFO:root:Bidder 18, loss: 32097242710016.0
INFO:root:Bidder 19, loss: 0.0
INFO:root:Bidder 20, loss: 2084692688896.0
INFO:root:Bidder 21, loss: 555981144064.0
INFO:root:Bidder 22, loss: 13665226981376.0
INFO:root:Bidder 23, loss: 11499528192.0
INFO:root:Bidder 24, 